In [86]:
import os
import sys

import numpy as np
import pandas as pd
import math

from trackml.dataset import load_event
from trackml.randomize import shuffle_hits
from trackml.score import score_event

import collections as coll

from sklearn.neighbors import KDTree

sys.path.append("../..")
import merge as merge
import extension as ext
import track_score as scor
import zroutlier as zro


In [2]:
TRAIN_PATH = '../../../input/train_1'
event_id = 1029

In [3]:
CSV_PATH = '../../'
#event_1003_labels_train_helix1_phase1_dbscan1_premerge.csv
CSV_FILENAME_PREFIX = 'event_' + str(event_id) + '_labels_train_'
#CSV_HELIX1 = CSV_PATH + 'first_2_event_1003_labels_train_helix1_shift_5_6_m6_m9.csv'
#CSV_HELIX2 = CSV_PATH + 'second_event_1003_labels_train_helix1.csv'
#CSV_HELIX3 = CSV_PATH + 'third_2_event_1003_labels_train_helix4.csv'
#CSV_HELIX4 = CSV_PATH + 'fourth_event_1003_labels_train_helix2.csv'
CSV_FILENAME_PREFIX = 'event_' + str(event_id) + '_labels_train_helix1_phase1_dbscan'
CSV_HELIX1 = CSV_PATH + CSV_FILENAME_PREFIX + '1_processed.csv'
CSV_HELIX2 = CSV_PATH + CSV_FILENAME_PREFIX + '2_processed.csv'
CSV_HELIX3 = CSV_PATH + CSV_FILENAME_PREFIX + '3_processed.csv'
CSV_HELIX4 = CSV_PATH + CSV_FILENAME_PREFIX + '4_processed.csv'
CSV_HELIX5 = CSV_PATH + CSV_FILENAME_PREFIX + '5_processed.csv'
CSV_HELIX6 = CSV_PATH + CSV_FILENAME_PREFIX + '6_processed.csv'
CSV_HELIX7 = CSV_PATH + CSV_FILENAME_PREFIX + '7_processed.csv'
CSV_HELIX8 = CSV_PATH + CSV_FILENAME_PREFIX + '8_processed.csv'
CSV_HELIX9 = CSV_PATH + CSV_FILENAME_PREFIX + '9_processed.csv'
CSV_HELIX10 = CSV_PATH + CSV_FILENAME_PREFIX + '10_processed.csv'
CSV_HELIX11 = CSV_PATH + CSV_FILENAME_PREFIX + '11_processed.csv'
CSV_HELIX12 = CSV_PATH + CSV_FILENAME_PREFIX + '12_processed.csv'


In [ ]:
CSV_PATH = '../../best_csvs12/'
CSV_PATH2 = '../../'
#event_1003_labels_train_helix1_phase1_dbscan1_premerge.csv
CSV_FILENAME_PREFIX = 'event_' + str(event_id) + '_labels_train_helix1_phase2_dbscan'
CSV_HELIX1 = CSV_PATH + CSV_FILENAME_PREFIX + '1_premerge.csv'
CSV_HELIX2 = CSV_PATH + CSV_FILENAME_PREFIX + '2_premerge.csv'
CSV_HELIX3 = CSV_PATH + CSV_FILENAME_PREFIX + '3_premerge.csv'
CSV_HELIX4 = CSV_PATH + CSV_FILENAME_PREFIX + '4_premerge.csv'

In [4]:

event_prefix = 'event00000' + str(event_id)
hits, cells, particles, truth = load_event(os.path.join(TRAIN_PATH, event_prefix))

mem_bytes = (hits.memory_usage(index=True).sum() 
             + cells.memory_usage(index=True).sum() 
             + particles.memory_usage(index=True).sum() 
             + truth.memory_usage(index=True).sum())
print('{} memory usage {:.2f} MB'.format(event_prefix, mem_bytes / 2**20))

event000001029 memory usage 16.61 MB


In [5]:
#labels_hough = pd.read_csv(CSV_HOUGH).label.values
labels_helix1 = pd.read_csv(CSV_HELIX1).label.values
labels_helix2 = pd.read_csv(CSV_HELIX2).label.values
labels_helix3 = pd.read_csv(CSV_HELIX3).label.values
labels_helix4 = pd.read_csv(CSV_HELIX4).label.values
labels_helix5 = pd.read_csv(CSV_HELIX5).label.values
labels_helix6 = pd.read_csv(CSV_HELIX6).label.values
labels_helix7 = pd.read_csv(CSV_HELIX7).label.values
labels_helix8 = pd.read_csv(CSV_HELIX8).label.values
labels_helix9 = pd.read_csv(CSV_HELIX9).label.values
labels_helix10 = pd.read_csv(CSV_HELIX10).label.values
labels_helix11 = pd.read_csv(CSV_HELIX11).label.values
labels_helix12 = pd.read_csv(CSV_HELIX12).label.values

In [6]:
def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits.hit_id.values, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission


In [9]:
def display_score(message, event_id, labels, hits):
    submission = create_one_event_submission(event_id, hits, labels)
    score = score_event(truth, submission)
    print(message + ': ' + str(score))

In [ ]:
#labels_hough_filter = merge.remove_outliers(labels_hough, hits, print_counts=True)
labels_helix1_filter = merge.remove_outliers(labels_helix1, hits, print_counts=True)
labels_helix2_filter = merge.remove_outliers(labels_helix2, hits, print_counts=True)
#labels_helix3_filter = merge.remove_outliers(labels_helix3, hits, print_counts=True)
#labels_helix4_filter = merge.remove_outliers(labels_helix4, hits, print_counts=True)
#labels_helix5_filter = merge.remove_outliers(labels_helix5, hits, print_counts=True)
#labels_helix6_filter = merge.remove_outliers(labels_helix6, hits, print_counts=True)

In [31]:

hits['r'] = np.sqrt(hits.x**2+hits.y**2)
hits['zr'] = hits['z'] / hits['r']

In [243]:
labels_merged = heuristic_merge_tracks2(labels_helix1, labels_helix2, overwrite_limit=4, print_summary=True)
display_score('Merged1 score', event_id, labels_merged, hits)
#(labels_merged, _) = merge.remove_small_tracks(labels_merged, smallest_track_size=2)
labels_merged = heuristic_merge_tracks2(labels_merged, labels_helix3, overwrite_limit=4, print_summary=False)
display_score('Merged2 score', event_id, labels_merged, hits)
#(labels_merged, _) = merge.remove_small_tracks(labels_merged, smallest_track_size=2)
labels_merged = heuristic_merge_tracks2(labels_merged, labels_helix4, overwrite_limit=4, print_summary=False)
display_score('Merged3 score', event_id, labels_merged, hits)
(labels_merged, _) = merge.remove_small_tracks(labels_merged, smallest_track_size=3)
labels_merged = heuristic_merge_tracks2(labels_merged, labels_helix5, overwrite_limit=4, print_summary=False)
display_score('Merged4 score', event_id, labels_merged, hits)
#(labels_merged, _) = merge.remove_small_tracks(labels_merged, smallest_track_size=2)
labels_merged = heuristic_merge_tracks2(labels_merged, labels_helix6, overwrite_limit=4, print_summary=False)
display_score('Merged5 score', event_id, labels_merged, hits)
#(labels_merged, _) = merge.remove_small_tracks(labels_merged, smallest_track_size=3)
#(labels_merged, _) = merge.remove_small_tracks(labels_merged, smallest_track_size=2)
labels_merged = heuristic_merge_tracks2(labels_merged, labels_helix7, overwrite_limit=4, print_summary=False)
display_score('Merged6 score', event_id, labels_merged, hits)
(labels_merged, _) = merge.remove_small_tracks(labels_merged, smallest_track_size=3)
labels_merged = heuristic_merge_tracks2(labels_merged, labels_helix8, overwrite_limit=4, print_summary=False)
display_score('Merged7 score', event_id, labels_merged, hits)
#(labels_merged, _) = merge.remove_small_tracks(labels_merged, smallest_track_size=2)
labels_merged = heuristic_merge_tracks2(labels_merged, labels_helix9, overwrite_limit=4, print_summary=False)
display_score('Merged8 score', event_id, labels_merged, hits)
#(labels_merged, _) = merge.remove_small_tracks(labels_merged, smallest_track_size=2)
labels_merged = heuristic_merge_tracks2(labels_merged, labels_helix10, overwrite_limit=4, print_summary=False)
display_score('Merged9 score', event_id, labels_merged, hits)
(labels_merged, _) = merge.remove_small_tracks(labels_merged, smallest_track_size=3)
labels_merged = heuristic_merge_tracks2(labels_merged, labels_helix11, overwrite_limit=4, print_summary=False)
display_score('Merged10 score', event_id, labels_merged, hits)
#(labels_merged, _) = merge.remove_small_tracks(labels_merged, smallest_track_size=3)
#(labels_merged, _) = merge.remove_small_tracks(labels_merged, smallest_track_size=2)
labels_merged = heuristic_merge_tracks2(labels_merged, labels_helix12, overwrite_limit=4, print_summary=False)
display_score('Merged11 score', event_id, labels_merged, hits)
#(labels_merged, _) = merge.remove_small_tracks(labels_merged, smallest_track_size=2)
# Default: 0.64340, 0.64919, 0.65539, 0.65766, 0.65902, 0.66876, 0.66927, 0.66985, 0.67088, 0.67057, 0.67092
# overw_3: 0.64341, 0.64908, 0.65535, 0.65735, 0.65830, 0.66793, 0.66850, 0.66918, 0.67038, 0.67022, 0.67051
# longest: 0.63425, 0.63635, 0.63783, 0.63932, 0.63884, 0.65100, 0.64945, 0.64728, 0.65008, 0.64839, 0.65360
# longst2: 0.64025, 0.64245, 0.64172, 0.64383, 0.64109, 0.65684, 0.65461, 0.65163, 0.65430, 0.65111, 0.65831
# nosplit: 0.64298, 0.64868, 0.65475, 0.65720, 0.65837, 0.66846, 0.66895, 0.66982, 0.67101, 0.67064, 0.67167
# nosplt2: 0.64293, 0.64866, 0.65473, 0.65725, 0.65841, 0.66851, 0.66901, 0.66982, 0.67101, 0.67064, 0.67173
# scor1:   0.64083, 0.64277, 0.64301, 0.64526, 0.64166, 0.65782, 0.65691, 0.64359, 0.65646, 0.65365, 0.66034
# scor2:   BAD, 0.64800 final score
# Mknew:   0.64279, 0.64853, 0.65459, 0.65711, 0.65828, 0.66837, 0.66887, 0.66968, 0.67087, 0.67050, 0.67159
# mkovw:   ---> end is 0.67079
# mkovw+2: 0.64303, 0.64877, 0.65483, 0.65735, 0.65852, 0.66861, 0.66911, 0.66992, 0.67111, 0.67074, 0.67183
# current: 0.64293, 0.64866, 0.65473, 0.65725, 0.65841, 0.66851, 0.66901, 0.66982, 0.67101, 0.67064, 0.67173
# ovw_out: 0.64329, 0.64943, 0.65561, 0.65847, 0.65970, 0.67055, 0.67106, 0.67192, 0.67311, 0.67310, 0.67468
# outlr2:  0.64329, 0.64943, 0.65561, 0.65847, 0.65970, 0.67051, 0.67102, 0.67188, 0.67303, 0.67303, 0.67461
# --> look at next: new track creations from little overlap(0) (most frequent), non-0 (next), >20 (least frequent)
# +0trkeq: 0.64302, 0.64916, 0.65538, 0.65834, 0.65956, 0.67059, 0.67110, 0.67199, 0.67314, 0.67315, 0.67472
# +0trk+1: 0.64293, 0.64914, 0.65530, 0.65838, 0.65961, 0.67057, 0.67105, 0.67194, 0.67315, 0.67316, 0.67473
# +0trk+2: 0.64286, 0.64908, 0.65522, 0.65837, 0.65953, 0.67059, 0.67107, 0.67196, 0.67317, 0.67318, 0.67475
# +0trk+3: 0.64279, 0.64905, 0.65517, 0.65846, 0.65967, 0.67076, 0.67128, 0.67217, 0.67311, 0.67312, 0.67456
# +0trk++5: 0.64254, 0.64880, 0.65486, 0.65801, 0.65924, 0.67045, 0.67097, 0.67198, 0.67321, 0.67322, 0.67466
# +0trk++3: 0.64293, 0.64914, 0.65525, 0.65831, 0.65953, 0.67053, 0.67101, 0.67190, 0.67311, 0.67312, 0.67469
# +0trk++2: 0.64293, 0.64914, 0.65530, 0.65838, 0.65961, 0.67057, 0.67105, 0.67194, 0.67315, 0.67316, 0.67473
# +0trk++: 0.64302, 0.64916, 0.65538, 0.65834, 0.65956, 0.67059, 0.67110, 0.67199, 0.67314, 0.67315, 0.67472
# ->+rem:  0.64302, 0.64916, 0.65538, 0.65834, 0.65956, 0.67059, 0.67110, 0.67199, 0.67314, 0.67315, 0.67472
# ->+rem2: 0.64302, 0.64916, 0.65538, 0.65814, 0.65936, 0.67035, 0.67086, 0.67205, 0.67320, 0.67323, 0.67495
# ->+rem2i: 0.64302, 0.64916, 0.65538, 0.65816, 0.65939, 0.67044, 0.67095, 0.67202, 0.67318, 0.67319, 0.67492
# ->+rmhlf: 0.64302, 0.64916, 0.65538, 0.65834, 0.65956, 0.67062, 0.67113, 0.67202, 0.67318, 0.67319, 0.67492
# ->fix0:   0.64378, 0.64992, 0.65610, 0.65897, 0.66019, 0.67109, 0.67160, 0.67245, 0.67362, 0.67363, 0.67535
# ->fix0+1: 0.64410, 0.65024, 0.65643, 0.65929, 0.66052, 0.67143, 0.67190, 0.67276, 0.67393, 0.67394, 0.67567
# ->fix0+2: 0.64437, 0.65057, 0.65676, 0.65962, 0.66080, 0.67170, 0.67217, 0.67304, 0.67420, 0.67422, 0.67590
### ->fix0+3: 0.64429, 0.65050, 0.65669, 0.65962, 0.66080, 0.67179, 0.67226, 0.67312, 0.67434, 0.67435, 0.67597
# ->fix0+4: 0.64421, 0.65032, 0.65652, 0.65942, 0.66053, 0.67144, 0.67192, 0.67278, 0.67395, 0.67384, 0.67553
# +other0:  0.64436, 0.65058, 0.65672, 0.65977, 0.66124, 0.67180, 0.67235, 0.67307, 0.67436, 0.67433, 0.67589
# +other1:  0.64354, 0.64974, 0..... 0.67569
# +other2:  0.64318, 0.64942, 0.65548, 0.65869, 0.65994, 0.67096, 0.67150, 0.67233, 0.67350, 0.67354, 0.67535
# +other3:  0.64300, 0.64926, 0.65536, 0.65897, 0.66003, 0.67087, 0.67142, 0.67223, 0.67339, 0.67342, 0.67536
# +othrunl: 0.64237, 0.64872, 0.65521, 0.65909, ..., 0.67542
# rem_dbl:  0.64486, 0.65110, 0.65737, 0.66036, 0.66181, 0.67233, 0.67280, 0.67360, 0.67475, 0.67474, 0.67588
# dblnon0:  0.64466, 0.65086, 0.65708, 0.65992, 0.66134, 0.67209, 0.67264, 0.67333, 0.67463, 0.67460, 0.67614
# 3waym1:   0.64466, 0.65091, 0.65717, 0.66001, 0.66145, 0.67220, 0.67276, 0.67344, 0.67474, 0.67471, 0.67623
# 3waym2:   0.64481, 0.65106, 0.65739, 0.66023, 0.66167, 0.67239, 0.67294, 0.67363, 0.67493, 0.67490, 0.67642
# 3waym2+1:   0.64487, 0.65113, 0.65746, 0.66036, 0.66186, 0.67266, 0.67315, 0.67384, 0.67508, 0.67505, 0.67649
# 3waym2+2:   0.64487, 0.65113, 0.65746, 0.66036, 0.66186, 0.67298, 0.67320, 0.67390, 0.67513, 0.67511, 0.67659
# 3waym2+3:   0.64503, 0.65129, 0.65764, 0.66055, 0.66204, 0.67308, 0.67331, 0.67401, 0.67529, 0.67526, 0.67680
# 3waym2+4:   0.64509, 0.65126, 0.65760, 0.66055, 0.66204, 0.67285, 0.67308, 0.67377, 0.67499, 0.67497, 0.67652
# 3waym5+1: 0.64505, 0.65135, 0.65770, 0.66079, 0.66225, 0.67329, 0.67358, 0.67421, 0.67555, 0.67553, 0.67715
# 3waym5+2: 0.64505, 0.65135, 0.65771, 0.66069, 0.66192, 0.67296, 0.67325, 0.67382, 0.67516, 0.67513, 0.67678
# 3waym5+3: 0.64458, 0.65091, 0.65726, 0.66025, 0.66147, 0.67246, 0.67275, 0.67331, 0.67466, 0.67458, 0.67623
# 3waym5a1: 0.65605, 0.65136, 0.65771, 0.66073, 0.66222, 0.67327, 0.67352, 0.67415, 0.67549, 0.67546, 0.67700
# 3waym5a2: 0.64505, 0.65136, 0.65777, 0.66075, 0.66201, 0.67305, 0.67330, 0.67388, 0.67521, 0.67519, 0.67675
# 3waym5a3: 0.64478, 0.65092, 0.65733, 0.66031, 0.66157, 0.67256, 0.67281, 0.67339, 0.67472, 0.67460, 0.67617
# 3waym510: 0.64505, 0.65137, 0.65772, 0.66080, 0.66226, 0.67330, 0.67359, 0.67422, 0.67555, 0.67552, 0.67714
# 3waym511: 0.64512, 0.65140, 0.65776, 0.66084, 0.66230, 0.67334, 0.67363, 0.67426, 0.67559, 0.67556, 0.67719
# 3wayx511: 0.64517, 0.65145, 0.65775, 0.66082, 0.66230, 0.67334, 0.67354, 0.67417, 0.67550, 0.67552, 0.67715
# 3waym512: 0.64512, 0.65141, 0.65776, 0.66084, 0.66230, 0.67334, 0.67363, 0.67426, 0.67559, 0.67556, 0.67719
# 3waym514: 0.64394, 0.65010, 0.65763, 0.66075, 0.66255, 0.67420, 0.67501, 0.67577, 0.67774, 0.67812, 0.68027
# 3waym515: 0.64456, 0.65085, 0.65789, 0.66111, 0.66281, 0.67473, 0.67556, 0.67636, 0.67816, 0.67859, 0.68064
### 3waym516: 0.64489, 0.65136, 0.65832, 0.66134, 0.66327, 0.67514, 0.67592, 0.67657, 0.67850, 0.67886, 0.68098
# 3waym517: 0.64498, 0.65120, 0.65820, 0.66123, 0.66289, 0.67466, 0.67543, 0.67634, 0.67812, 0.67806, 0.68030
## 3waym516, rem 2-track hits after each 3 merges, final score: 0.68106

Simple replacement of unclassified hits: 209
Similar tracks (no-op): 4010
New track creations from little overlap(0): 432
New track creations from huge tracks(>20): 8
Test for new track creations from little overlap(non-0): 16
--> Lengthen longest overlap instead: 10
  --> And clear 2nd longest track: 4
Skip extension due to too little overlap: 111
Multiple non-trivial tracks: 1199
--> of which partial track ID 0 hits were updated: 1601
--> of which partial track ID non-0 hits were updated: 0
--> of which partial track ID non-0 hits were skipped: 0
--> of which outliers were overwritten: 145
Tracks to be lengthened: 1343
--> of which track ID 0 hits were updated: 3199
--> from which new tracks were created instead: 0
--> of which labels2 unique track lengths were: [ 5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 25]
Merged1 score: 0.644889382557
Merged2 score: 0.651335948918
Merged3 score: 0.658322110111
Merged4 score: 0.661137585052
Merged5 score: 0.66315586132
Merged6 score: 0.675132

In [239]:
def heuristic_merge_tracks2(labels1, labels2, overwrite_limit=4, print_summary=False):
    return heuristic_merge_tracks3(labels1, labels2, hits, overwrite_limit=overwrite_limit, print_summary=print_summary)

def heuristic_merge_tracks3(labels1, labels2, hits, overwrite_limit=4, print_summary=True):
    """ Merge tracks from two arrays of track labels.

    Merges are handled as follows:
     - tracks from labels2 are identified and searched
     - for each track from labels2:
       - use track directly if no conflict with any tracks from labels1
       - skip if labels1 already contains the same track of equal (or longer) length
       - otherwise, if there are potentially multiple conflicting tracks from labels1
         - if labels1 only contains a single track ID, as well as un-classified (0) hits,
           re-assign '0' track ID to labels1 track ID (i.e. lengthen the track)
         - otherwise, labels1 contains multiple non-zero track IDs
           - replace any track ID 0 occurrences with the longest labels1 track ID
           - replace any occurrences of short (len <= 3) labels1 tracks with the longest labels1 track ID

    Parameters:
     - labels1: np array of labels, each entry represents a hit, the value represents the
       track ID that hit is assigned to. This should be considered the 'higher-quality' of
       the two input labels
     - labels2: np array of secondary labels, whose tracks should be merged into labels1

    Returns: The merged array of labeled tracks.
    """
    labels_merged = np.copy(labels1)
    labels_merged = merge.renumber_labels(labels_merged)
    max_track = np.amax(labels_merged)
    labels2[labels2 != 0] = labels2[labels2 != 0] + max_track
    trks2 = np.unique(labels2)
    count1 = 0
    count2 = 0
    count3 = 0
    count4 = 0
    count4_len = []
    count5 = 0
    count6 = 0
    count7 = 0
    count8 = 0
    count9 = 0
    count10 = 0
    count11 = 0
    count12 = 0
    count13 = 0
    count14 = 0
    count15 = 0
    count16 = 0
    for trk2 in trks2:
        if trk2 == 0:
            continue
        trk2_ix = np.where(labels2 == trk2)[0]
        trk2_length = len(trk2_ix)
        if trk2_length < 2:
            continue
        trk1_val = labels_merged[trk2_ix]
        #print('trk2: ' + str(trk2) + ', label1: ' + str(trk1_val))
        trk1_uniq = np.unique(trk1_val)
        # Now we know which tracks from the 1st label overlap with the tracks from the 2nd label
        if len(trk1_uniq) == 1:
            if trk1_uniq[0] == 0:
                #print('Good candidate to replace!')
                # This track was not found by labels1, just directly use the
                # track from labels2.
                count1 = count1 + 1
                labels_merged[trk2_ix] = trk2
            else:
                # We found a track that is at least as long as the current
                # track in labels1. Nothing more needed, at least for now.
                # We could consider scenarios where the labels1 track contains
                # hits from 2 different tracks, where labels2 only has a
                # shorter single track. In this case, it may be good to split
                # the labels1 track into two pieces. However, this condition
                # would be very hard to detect, for now we want to favour
                # longer tracks whenever possible.
                #print('Same track found, skipping...')
                count2 = count2 + 1
        else:
            found_tracks = 0
            # Get counts for all identified tracks from labels1 that match trk2
            trk1_counts = coll.Counter(trk1_val).most_common(len(trk1_uniq))
            longest_track_id = trk1_counts[0][0]
            longest_track_count = trk1_counts[0][1]
            second_track_id = trk1_counts[1][0]
            second_track_count = trk1_counts[1][1]
            # If longest track in labels1 was 0, create a new track, but only
            # from free hits, or from small tracks. Also, if there is not
            # enough overlap (less than half the hits overlap), also create
            # a new track.
            if longest_track_id == 0:
                count5 = count5 + 1
                longest_track_id = trk2
                # See if we should instead lengthen the longest non-zero track
                if len(trk1_uniq) == 2:
                    test_track_ix = np.where(labels_merged == second_track_id)[0]
                    if len(test_track_ix) <= (second_track_count + 3):
                        outliers1 = zro.find_track_outliers_zr(second_track_id, labels_merged, hits, find_all=True)
                        labelx = np.copy(labels_merged)
                        labelx[trk2_ix] = second_track_id
                        outliers2 = zro.find_track_outliers_zr(second_track_id, labelx, hits, find_all=True)
                        if len(outliers2) <= len(outliers1):
                            longest_track_id = second_track_id
                            longest_track_count = second_track_count
            elif (trk2_length > 20) or (longest_track_count > 20):
                count9 = count9 + 1
                longest_track_id = trk2
            elif (trk2_length > 6) and (longest_track_count < int(trk2_length/2)) and second_track_id != 0:
                # Try to avoid creating crossed tracks, do not lengthen existing track if not
                # enough overlap.
                count10 = count10 + 1
                trk1a = np.where(labels_merged == longest_track_id)[0]
                if longest_track_count + 3 >= len(trk1a):
                    #print('LIAM: Top 2 tracks, new: ' + str(trk2_length) + ', len1: '  + str(longest_track_count) + ', len1a: ' + str(len(trk1a)) + ', len2: ' + str(second_track_count) + ', len2a: ' + str(len(trk2a)))
                    # Lengthen the longest track, it's fully contained by our new/proposed track.
                    # Reset 2nd longest track if mostly contained in new/proposed track.
                    count14 = count14 + 1
                    trk2a = np.where(labels_merged == second_track_id)[0]
                    if second_track_count + 1 >= len(trk2a):
                        count15 = count15 + 1
                        labels_merged[trk2a] = longest_track_id
                else:
                    # Not much overlap, start a new track to avoid hurting existing tracks.
                    longest_track_id = trk2
            else:
                # If the old track had too many hits not part of the new/proposed track, do
                # not lengthen it - that may lose majority. Better to start a new track.
                trk1a = np.where(labels_merged == longest_track_id)[0]
                if longest_track_count + 6 < len(trk1a):
                    count16 = count16 + 1
                    longest_track_id = trk2

            for trk1 in trk1_uniq:
                if trk1 == 0:
                    continue
                trk1_ix = np.where(labels_merged == trk1)[0]
                if len(trk1_ix) > 1:
                    found_tracks = found_tracks + 1
            if found_tracks > 1:
                #print('Found ' + str(found_tracks) + ' non-trivial tracks.')
                count3 = count3 + 1
                # If there are un-classified hits, assign those to the track
                # ID with the most hits.
                for label_ix in trk2_ix:
                    if labels_merged[label_ix] == 0:
                        labels_merged[label_ix] = longest_track_id
                        count6 = count6 + 1

                # If there are tracks of length 2 or less, and one or both
                # of those hits are included in the target track, re-assign
                # those matching the labels2 track to the most common
                # original track ID.
                for trk1_count in trk1_counts:
                    #if trk1_count[1] <= overwrite_limit:
                    trk1_count_ix = np.where(labels_merged == trk1_count[0])[0]
                    if len(trk1_count_ix) <= overwrite_limit:
                        outliers = zro.find_track_outliers_zr(trk2, labels2, hits, find_all=True)
                        for label_ix in trk2_ix:
                            if labels_merged[label_ix] == trk1_count[0] and label_ix in outliers:
                                count13 = count13 + 1
                            elif labels_merged[label_ix] == trk1_count[0]:# and label_ix not in outliers:
                                labels_merged[label_ix] = longest_track_id
                                count7 = count7 + 1
                    else:
                        outliers = zro.find_track_outliers_zr(trk1_count[0], labels_merged, hits, find_all=True)
                        for label_ix in trk2_ix:
                            if labels_merged[label_ix] == trk1_count[0] and label_ix in outliers:
                                labels_merged[label_ix] = longest_track_id
                                count12 = count12 + 1

            else:
                # Only the track ID, as well as track ID 0, were found in labels1.
                # Replace any occurrences of ID 0 with the labels1 track ID.
                count4 = count4 + 1
                count4_len.append(len(trk2_ix))
                # If there are un-classified hits, assign those to the track
                # ID with the most hits (lengthens the track).
                for label_ix in trk2_ix:
                    if labels_merged[label_ix] == 0:
                        labels_merged[label_ix] = longest_track_id
                        count8 = count8 + 1

    if print_summary:
        print('Simple replacement of unclassified hits: ' + str(count1))
        print('Similar tracks (no-op): ' + str(count2))
        print('New track creations from little overlap(0): ' + str(count5))
        print('New track creations from huge tracks(>20): ' + str(count9))
        print('Test for new track creations from little overlap(non-0): ' + str(count10))
        print('--> Lengthen longest overlap instead: ' + str(count14))
        print('  --> And clear 2nd longest track: ' + str(count15))
        print('Skip extension due to too little overlap: ' + str(count16))
        print('Multiple non-trivial tracks: ' + str(count3))
        print('--> of which partial track ID 0 hits were updated: ' + str(count6))
        print('--> of which partial track ID non-0 hits were updated: ' + str(count7))
        print('--> of which partial track ID non-0 hits were skipped: ' + str(count13))
        print('--> of which outliers were overwritten: ' + str(count12))
        print('Tracks to be lengthened: ' + str(count4))
        print('--> of which track ID 0 hits were updated: ' + str(count8))
        print('--> from which new tracks were created instead: ' + str(count11))
        noises = np.unique(np.asarray(count4_len))
        print('--> of which labels2 unique track lengths were: ' + str(noises))

    return labels_merged

In [ ]:
labels_7_extend = np.copy(labels_hough)
print(len(labels_7_extend))
one_submission = create_one_event_submission(event_id, hits, labels_7_extend)
score = score_event(truth, one_submission)
print("Extension %d loop %d score for event %d: %.8f" % (ix,i+1,event_id, score))
track_extension_limits = [0.02, 0.04, 0.06, 0.08, 0.10]
for ix, limit in enumerate(track_extension_limits):
    labels_7_extend = extend_labels(ix, labels_7_extend, hits, do_swap=(ix%2==1), limit=(limit))
    one_submission = create_one_event_submission(event_id, hits, labels_7_extend)
    score = score_event(truth, one_submission)
    print("Extension %d score for event %d: %.8f" % (ix,event_id, score))

In [ ]:
EXTENSION_ATTEMPT = 8
EXTENSION_LIMIT_START = 0.03
EXTENSION_LIMIT_INTERVAL = 0.005


l1a = np.copy(labels_helix1)
l2a = np.copy(labels_helix2)
l3a = np.copy(labels_helix3)
l4a = np.copy(labels_helix4)
for i in range(EXTENSION_ATTEMPT):
    limit = EXTENSION_LIMIT_START + EXTENSION_LIMIT_INTERVAL*i
    l1a = ext.extend_labels(i, l1a, hits, do_swap=i%2==1, limit=(limit))
    l2a = ext.extend_labels(i, l2a, hits, do_swap=i%2==1, limit=(limit))
    l3a = ext.extend_labels(i, l3a, hits, do_swap=i%2==1, limit=(limit))
    l4a = ext.extend_labels(i, l4a, hits, do_swap=i%2==1, limit=(limit))

l1a = merge.renumber_labels(l1a)
l2a = merge.renumber_labels(l2a)
l3a = merge.renumber_labels(l3a)
l4a = merge.renumber_labels(l4a)

seed_length = 5
my_volumes = [7, 8, 9]
l1a = merge.filter_invalid_tracks(l1a, hits, my_volumes, seed_length)
l1a = merge.renumber_labels(l1a)
l2a = merge.filter_invalid_tracks(l2a, hits, my_volumes, seed_length)
l2a = merge.renumber_labels(l2a)
l3a = merge.filter_invalid_tracks(l3a, hits, my_volumes, seed_length)
l3a = merge.renumber_labels(l3a)
l4a = merge.filter_invalid_tracks(l4a, hits, my_volumes, seed_length)
l4a = merge.renumber_labels(l4a)

# Perform sophisticated outlier removal, duplicate-z removal, slope-based removal
l1a = merge.remove_outliers(l1a, hits, print_counts=False)
l2a = merge.remove_outliers(l2a, hits, print_counts=False)
l3a = merge.remove_outliers(l3a, hits, print_counts=False)
l4a = merge.remove_outliers(l4a, hits, print_counts=False)

In [ ]:
labels_merged = merge.heuristic_merge_tracks(l1a, l2a)
#labels_merged = merge.heuristic_merge_tracks(labels_merged, l3a)
#labels_merged = merge.heuristic_merge_tracks(labels_merged, l4a)
labels_merged2 = merge.heuristic_merge_tracks(l3a, l4a)
labels_merged = merge.heuristic_merge_tracks(labels_merged, labels_merged2)
#labels_merged = merge.heuristic_merge_tracks(labels_merged, labels_helix4_filter)
#labels_merged = merge.heuristic_merge_tracks(labels_merged, labels_helix5_filter)
#labels_merged = merge.heuristic_merge_tracks(labels_merged, labels_helix6_filter)
one_submission = create_one_event_submission(event_id, hits, labels_merged)
score = score_event(truth, one_submission)
print("Merged score for event %d: %.8f" % (event_id, score))

In [ ]:
def hack_one_extend(labels, hits, truth):
    angle=82
    limit=0.04
    df = hits.copy(deep=True)
    df['track_id'] = labels.tolist()
    df = df.assign(d = np.sqrt( df.x**2 + df.y**2 + df.z**2 ))
    df = df.assign(r = np.sqrt( df.x**2 + df.y**2))
    df = df.assign(arctan2 = np.arctan2(df.z, df.r))
    df = _one_cone_slice(df, truth, angle, 1, limit)
    return df.track_id.values

def extend(iter, df, do_swap=False, limit=0.04):
    if do_swap:
        df = df.assign(x = -df.x)
        df = df.assign(y = -df.y)

    df = df.assign(d = np.sqrt( df.x**2 + df.y**2 + df.z**2 ))
    df = df.assign(r = np.sqrt( df.x**2 + df.y**2))
    df = df.assign(arctan2 = np.arctan2(df.z, df.r))


    for angle in range(-90,90,1):

        print ('\r%d %f '%(iter,angle), end='',flush=True)
        df1 = df.loc[(df.arctan2>(angle-1.0)/180*np.pi) & (df.arctan2<(angle+1.0)/180*np.pi)]

        num_hits = len(df1)
        # Dynamically adjust the delta based on how many hits are found
        if num_hits > 2000:
            df = _one_cone_slice(df, angle-0.6, 0.4, limit)
            df = _one_cone_slice(df, angle-0.2, 0.4, limit)
            df = _one_cone_slice(df, angle+0.2, 0.4, limit)
            df = _one_cone_slice(df, angle+0.6, 0.4, limit)
        else:
            df = _one_cone_slice(df, angle, 1, limit)
           
    return df

def extend_labels(iter, labels, hits, do_swap=False, limit=0.04):
    df = hits.copy(deep=True)
    df['track_id'] = labels.tolist()
    return extend(iter, df, do_swap, limit).track_id.values

def _one_cone_slice(df, truth, angle, delta_angle, limit=0.04, num_neighbours=18):

    df1 = df.loc[(df.arctan2>(angle - delta_angle)/180*np.pi) & (df.arctan2<(angle + delta_angle)/180*np.pi)]

    min_num_neighbours = len(df1)
    if min_num_neighbours < 3: 
        return df

    hit_ids = df1.hit_id.values
    x,y,z = df1.as_matrix(columns=['x', 'y', 'z']).T
    r  = (x**2 + y**2)**0.5
    r  = r/1000
    a  = np.arctan2(y,x)
    c = np.cos(a)
    s = np.sin(a)
    tree = KDTree(np.column_stack([c,s,r]), metric='euclidean')

    track_ids = list(df1.track_id.unique())
    num_track_ids = len(track_ids)
    min_length=3

    
    for i in range(num_track_ids):
        p = track_ids[i]
        if p==0: continue

        idx = np.where(df1.track_id==p)[0]
        cur_track_len = len(idx)
        if cur_track_len<min_length: continue

        truth_ix = []
        for ii in idx:
            truth_ix.append(hit_ids[ii] - 1)
        tdf = truth.loc[truth_ix]
        truth_count = coll.Counter(tdf.particle_id.values).most_common(1)
        truth_particle_id = truth_count[0][0]
        #print('track: ' + str(p) + ', len: ' + str(len(idx)) + ', idx: ' + str(idx))
        #print('truth particle: ' + str(truth_particle_id) + ', count:' + str(truth_count[0][1]))
            
        if angle>0:
            idx = idx[np.argsort( z[idx])]
        else:
            idx = idx[np.argsort(-z[idx])]

## start and end points  ##
        idx0,idx1 = idx[0],idx[-1]
        a0 = a[idx0]
        a1 = a[idx1]
        r0 = r[idx0]
        r1 = r[idx1]
        c0 = c[idx0]
        c1 = c[idx1]
        s0 = s[idx0]
        s1 = s[idx1]

        da0 = a[idx[1]] - a[idx[0]]  #direction
        dr0 = r[idx[1]] - r[idx[0]]
        direction0 = np.arctan2(dr0,da0)

        da1 = a[idx[-1]] - a[idx[-2]]
        dr1 = r[idx[-1]] - r[idx[-2]]
        direction1 = np.arctan2(dr1,da1)

        ## extend start point
        ns = tree.query([[c0, s0, r0]], k=min(num_neighbours, min_num_neighbours), return_distance=False)
        ns = np.concatenate(ns)

        direction = np.arctan2(r0 - r[ns], a0 - a[ns])
        diff = 1 - np.cos(direction - direction0)
        ns = ns[(r0 - r[ns] > 0.01) & (diff < (1 - np.cos(limit)))]
        for n in ns:
            df_ix = hit_ids[n] - 1
            if truth.loc[df_ix, 'particle_id'] == truth_particle_id:
                print('Found valid start extension!')
            else:
                print('Found WRONG start extension!')
            old_track = df.loc[df_ix, 'track_id']
            if old_track == 0:
                df.loc[df_ix, 'track_id'] = p
            elif old_track != 0:
                # If the hit is already occupied by another track, only take ownership
                # of the hit if our track is longer than the current-occupying track.
                existing_track_len = len(np.where(df.track_id==old_track)[0])
                if cur_track_len > existing_track_len:
                    df.loc[df_ix, 'track_id'] = p
    

        ## extend end point
        ns = tree.query([[c1, s1, r1]], k=min(num_neighbours, min_num_neighbours), return_distance=False)
        ns = np.concatenate(ns)

        direction = np.arctan2(r[ns] - r1, a[ns] - a1)
        diff = 1 - np.cos(direction - direction1)
  
        ns = ns[(r[ns] - r1 > 0.01) & (diff < (1 - np.cos(limit)))]
        for n in ns:  
            df_ix = hit_ids[n] - 1
            if truth.loc[df_ix, 'particle_id'] == truth_particle_id:
                print('Found valid end extension!')
            else:
                print('Found WRONG end extension!')
            old_track = df.loc[df_ix, 'track_id']
            if old_track == 0:
                df.loc[df_ix, 'track_id'] = p
            elif old_track != 0:
                # If the hit is already occupied by another track, only take ownership
                # of the hit if our track is longer than the current-occupying track.
                existing_track_len = len(np.where(df.track_id==old_track)[0])
                if cur_track_len > existing_track_len:
                    df.loc[df_ix, 'track_id'] = p
      
    return df

In [ ]:
new_labels = hack_one_extend(labels_helix1, hits, truth)

In [ ]:
# Test pre-merge
#labels_helix1_filter = np.copy(labels_helix1)
#labels_helix2_filter = np.copy(labels_helix2)
#labels_helix3_filter = np.copy(labels_helix3)
#labels_helix4_filter = np.copy(labels_helix4)

labels_merged = merge.heuristic_merge_tracks(labels_helix1_filter, labels_helix2_filter)
labels_merged2 = merge.heuristic_merge_tracks(labels_helix3_filter, labels_helix4_filter)
labels_merged = merge.heuristic_merge_tracks(labels_merged, labels_merged2)
#one_submission = create_one_event_submission(event_id, hits, labels_merged)
#score = score_event(truth, one_submission)
#print("H1/H2 Merged score for event %d: %.8f" % (event_id, score))
#one_submission = create_one_event_submission(event_id, hits, labels_merged2)
#score = score_event(truth, one_submission)
#print("H3/H4 Merged score for event %d: %.8f" % (event_id, score))

#labels_merged = merge.heuristic_merge_tracks(labels_helix1_filter, labels_helix2_filter)
#labels_merged = merge.heuristic_merge_tracks(labels_merged, labels_helix3_filter)
#labels_merged = merge.heuristic_merge_tracks(labels_merged, labels_helix4_filter)

one_submission = create_one_event_submission(event_id, hits, labels_merged)
score = score_event(truth, one_submission)
print("Merged score for event %d: %.8f" % (event_id, score))